In [1]:
pip install -U transformers datasets accelerate peft -q

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'd:\Aventus3\venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [2]:
import transformers
import peft

print("Transformers version:", transformers.__version__)
print("PEFT version:", peft.__version__)

d:\Aventus3\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Transformers version: 4.51.3
PEFT version: 0.15.2


In [3]:
from peft import get_peft_model, LoraConfig, TaskType
from transformers import AutoModelForSequenceClassification, AutoTokenizer

model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
base_model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
config = LoraConfig(
    task_type=TaskType.SEQ_CLS,
    inference_mode=False,
    r=8,
    lora_alpha=16,
    lora_dropout=0.1,
    target_modules=["q_lin", "v_lin"],  # 👈 key fix
)


model = get_peft_model(base_model, config)
model.print_trainable_parameters()


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 739,586 || all params: 67,694,596 || trainable%: 1.0925


In [ ]:
from datasets import load_dataset

dataset = load_dataset("hendzh/PromptShield")
def tokenize_fn(example):
    return tokenizer(example["prompt"], truncation=True, padding="max_length", max_length=256)
tokenized = dataset.map(tokenize_fn, batched=True)


Map: 100%|██████████| 23516/23516 [00:04<00:00, 5240.46 examples/s]


: 

In [ ]:
from torch.utils.data import DataLoader
from transformers import TrainingArguments, Trainer

train_dataset = tokenized["train"].with_format("torch")
val_dataset = tokenized["validation"].with_format("torch")

from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./lora-distilbert",
    eval_strategy="epoch",  # Updated parameter name
    save_strategy="epoch",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    learning_rate=2e-4,
    load_best_model_at_end=True,
    logging_dir="./logs",
    report_to="none",  # Prevents wandb errors
)




trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
)
trainer.train()


C:\Users\incha\AppData\Local\Temp\ipykernel_30196\1330672771.py:25: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
d:\Aventus3\venv\lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss


In [ ]:
model.save_pretrained("./lora_adapter")
tokenizer.save_pretrained("./lora_adapter")